In [89]:
from bs4 import BeautifulSoup
import requests
import MeCab as mc
import json
import sqlite3
import requests
import records
import glob
import json
import urllib.parse
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import gensim
from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary
from gensim import corpora, matutils
from sklearn.model_selection import train_test_split
from sklearn import ensemble

In [10]:
# from homepage to IT page by BeautifulSoup
url = "https://news.yahoo.co.jp/"
home_r = requests.get(url)
home_bs = , ma(home_r.text, 'html.parser')

print(home_bs)

<!DOCTYPE html>
<html lang="ja"><head><title data-reactroot="">Yahoo!ニュース</title><meta content="text/html; charset=utf-8" data-reactroot="" http-equiv="Content-Type"/><meta content="text/css" data-reactroot="" http-equiv="Content-Style-Type"/><meta content="text/javascript" data-reactroot="" http-equiv="Content-Script-Type"/><meta content="ニュース,ヤフー,新着" data-reactroot="" name="keywords"/><meta content="Yahoo!ニュースは、新聞・通信社が配信するニュースのほか、映像、雑誌や個人の書き手が執筆する記事など多種多様なニュースを掲載しています。" data-reactroot="" name="description"/><meta content="noarchive" data-reactroot="" name="robots"/><meta content="telephone=no" data-reactroot="" name="format-detection"/><meta content="dVyTbsDzUrUAjCE1aCH9hKwZ8sOsCsvi2uhuBvasrp8" data-reactroot="" name="google-site-verification"/><meta content="https://s.yimg.jp/images/jpnews/cre/common/all/images/fbico_ogp_1200x630.png" data-reactroot="" property="og:image"/><meta content="Yahoo!ニュースは、新聞・通信社が配信するニュースのほか、映像、雑誌や個人の書き手が執筆する記事など多種多様なニュースを掲載しています。" data-reactroot="" proper

In [11]:
# 定义链接数据库，建造表，的关数
conn = None

def connect():
    global conn
    conn = sqlite3.connect('./yahoo.db')
    
def close():
    conn.close()
    
def create_table():
    conn.execute('DROP TABLE IF EXISTS news')
    conn.execute('''CREATE TABLE news(
    id               INTEGER PRIMARY KEY AUTOINCREMENT,
    CONTENT          TEXT,
    category         TEXT
    
    )''')
    
def load(content,category):
    conn.execute(
    'INSERT INTO news (CONTENT,category) VALUES (?,?)', (content,category))
    conn.commit()

In [14]:
connect()
create_table()


In [15]:
url = 'https://news.yahoo.co.jp/list/?c=computer&p='
url2 = 'https://news.yahoo.co.jp/list/?c=sports&p='
for i in range(51):
    it_all_link = url+str(i + 1)
#     print(it_all_link)
    it_r = requests.get(it_all_link)
    it_url_bs = BeautifulSoup(it_r.text, 'html.parser')
    title_table = it_url_bs.find("ul", class_="list")
    for dt in title_table.find_all('dt'):
        title= dt.text
#         print(title)
        load(title,'IT') 

for i in range(51):
    it_all_link2 = url2+str(i + 1)
#     print(it_all_link)
    it_r2 = requests.get(it_all_link2)
    it_url_bs2 = BeautifulSoup(it_r2.text, 'html.parser')
    title_table2 = it_url_bs.find("ul", class_="list")
    for dt in title_table2.find_all('dt'):
        title2= dt.text
#         print(title)
        load(title2,'SPORTS')      

In [16]:
close()

In [18]:
con = None
con = sqlite3.connect("./yahoo.db")
cursor = con.cursor()
res = cursor.execute("SELECT CONTENT FROM news")
bs = cursor.fetchall()
print(bs)

[('ポケモン 睡眠をエンタメ化',), ('ポケモン 新作ゲーム2本発表',), ('ポケモン DeNAと新作ゲーム',), ('海外で共感? スマホNG結婚式',), ('新iPod touch ゲームに重点',), ('新型iPod touch 突如発表',), ('スマホ商戦に異状 上位に逆風',), ('ビットコイン価格回復 背景は',), ('見城氏 日本国紀への批判一蹴',), ('林死刑囚長男 発信する理由',), ('賞贈呈式 作家が幻冬舎を批判',), ('かぶる傘 コラ画像のデマ拡散',), ('大統領と握手の作家 招待否定',), ('外資規制にITも 情報流出防ぐ',), ('配膳ロボット導入 現場の声は',), ('ハムスター動画で「虐待」',), ('米の華為排除 もろ刃の剣',), ('商談の良しあし ソフトが判定',), ('不謹慎狩り 続く不毛な連鎖',), ('華為スマホ出荷 最大24%減',), ('「パオチャン」が活動休止へ',), ('SNS炎上「防災訓練」が人気',), ('安全ピンで痴漢撃退過剰?賛否',), ('在宅で稼げる トラブルに注意',), ('荒野行動とPUBG 和解が成立',), ('NGT支配人Twitter開始に賛否',), ('FB偽アカ 半年で30億件超削除',), ('ネット弱い主婦 加害者にも',), ('政府のドローン 8割が中国製',), ('スパコン 京の後継は「富岳」',), ('AIが服コーデ提案 ゾゾ子会社',), ('社長のツイート 幻冬舎が謝罪',), ('華為の対応疑問 顧客離れ危機',), ('焼きそばパン 愛される秘密は',), ('ZOZO 実店舗も「ツケ払い」',), ('QRコードで扉制御へ 浅草線',), ('華為スマホ ドコモも予約停止',), ('SBとKDDI 華為スマホ発売延期',), ('偽医師がデマ発信 信頼どこに',), ('はあちゅう氏 アンチ無視せず',), ('満員電車 理想の動き方を分析',), ('ファーウェイ 独自OS投入か',), ('ゲーム依存 医師語る「怖さ」',), ('河野外相が謎ツイート 話題',), ('LINE Payの還元 1日で90億円',), ('eスポーツ ゲーム依存を助長?',), ('福岡空港 扉の暗証番号誤送

In [53]:
magachi = [mecab_analysis(text[0]) for text in bs]
magachi
# word_text = [w[0] for w in bs]

[['ポケモン', '睡眠', 'エン', 'タメ', '化'],
 ['ポケモン', '新作', 'ゲーム', '2', '本', '発表'],
 ['ポケモン', 'DeNA', '新作', 'ゲーム'],
 ['海外', '共感', '?', 'スマホ', 'NG', '結婚式'],
 ['iPod', 'touch', 'ゲーム', '重点'],
 ['新型', 'iPod', 'touch', '発表'],
 ['スマホ', '商戦', '異状', '上位', '逆風'],
 ['ビット', 'コイン', '価格', '回復', '背景'],
 ['見城', '氏', '日本国', '紀', '批判', '一蹴'],
 ['林', '死刑', '囚', '長男', '発信', '理由'],
 ['賞', '贈呈', '式', '作家', '幻冬舎', '批判'],
 ['傘', 'コラ', '画像', 'デマ', '拡散'],
 ['大統領', '握手', '作家', '招待', '否定'],
 ['外資', '規制', 'IT', '情報', '流出'],
 ['配膳', 'ロボット', '導入', '現場', '声'],
 ['ハムスター', '動画', '虐待'],
 ['米', '華', '為', '排除', 'もろ刃', '剣'],
 ['商談', '良しあし', 'ソフト', '判定'],
 ['不謹慎', '不毛', '連鎖'],
 ['華', '為', 'スマホ', '出荷', '最大', '24', '%', '減'],
 ['パオ', 'チャン', '活動', '休止'],
 ['SNS', '炎上', '防災', '訓練', '人気'],
 ['安全ピン', '痴漢', '撃退', '過剰', '?', '賛否'],
 ['在宅', 'トラブル', '注意'],
 ['荒野', '行動', 'PUBG', '和解', '成立'],
 ['NGT', '支配人', 'Twitter', '開始', '賛否'],
 ['FB', 'アカ', '半年', '30', '億', '件', '削除'],
 ['ネット', '主婦', '加害', '者'],
 ['政府', 'ローン', '8', '割', '中国', '製'],
 ['スパコン

In [25]:
def mecab_analysis(texts):
    t = mc.Tagger("")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞']:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [29]:

wgachi=mecab_analysis(" ".join(magachi))
# print(wgachi)

['ポケモン', '睡眠', 'エン', 'タメ', '化', 'ポケモン', '新作', 'ゲーム', '2', '本', '発表', 'ポケモン', 'DeNA', '新作', 'ゲーム', '海外', '共感', '?', 'スマホ', 'NG', '結婚式', 'iPod', 'touch', 'ゲーム', '重点', '新型', 'iPod', 'touch', '発表', 'スマホ', '商戦', '異状', '上位', '逆風', 'ビット', 'コイン', '価格', '回復', '背景', '見城', '氏', '日本国', '紀', '批判', '一蹴', '林', '死刑', '囚', '長男', '発信', '理由', '賞', '贈呈', '式', '作家', '幻冬舎', '批判', '傘', 'コラ', '画像', 'デマ', '拡散', '大統領', '握手', '作家', '招待', '否定', '外資', '規制', 'IT', '情報', '流出', '配膳', 'ロボット', '導入', '現場', '声', 'ハムスター', '動画', '虐待', '米', '華', '為', '排除', 'もろ刃', '剣', '商談', '良しあし', 'ソフト', '判定', '不謹慎', '不毛', '連鎖', '華', '為', 'スマホ', '出荷', '最大', '24', '%', '減', 'パオ', 'チャン', '活動', '休止', 'SNS', '炎', '上', '防災', '訓練', '人気', '安全ピン', '痴漢', '撃退', '過剰', '?', '賛否', '在宅', 'トラブル', '注意', '荒野', '行動', 'PUBG', '和解', '成立', 'NGT', '支配人', 'Twitter', '開始', '賛否', 'FB', 'アカ', '半年', '30', '億', '件', '削除', 'ネット', '主婦', '加害', '者', '政府', 'ローン', '8', '割', '中国', '製', 'スパコン', '京', '後継', '富岳', 'AI', '服', 'コーデ', '提案', 'ゾゾ', '子会社', '社長', 'ツイート', '幻冬舎', '謝罪', 

In [47]:
model = Word2Vec(magachi, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
sim = model.similarity('億','円')
sim


/home/r-ei/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


0.70552564

In [48]:
model.most_similar(positive='ポケモン')

/home/r-ei/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('料金', 0.31989383697509766),
 ('大野', 0.3098503351211548),
 ('修正', 0.2911008596420288),
 ('サムスン', 0.2857826352119446),
 ('返金', 0.2808879017829895),
 ('訓練', 0.2793143391609192),
 ('開設', 0.27524831891059875),
 ('アカデミー', 0.2723502516746521),
 ('JAL', 0.27004510164260864),
 ('S', 0.2680143713951111)]

In [51]:
dictionary = corpora.Dictionary(magachi)
print(dictionary.token2id)

{'エン': 0, 'タメ': 1, 'ポケモン': 2, '化': 3, '睡眠': 4, '2': 5, 'ゲーム': 6, '新作': 7, '本': 8, '発表': 9, 'DeNA': 10, '?': 11, 'NG': 12, 'スマホ': 13, '共感': 14, '海外': 15, '結婚式': 16, 'iPod': 17, 'touch': 18, '重点': 19, '新型': 20, '上位': 21, '商戦': 22, '異状': 23, '逆風': 24, 'コイン': 25, 'ビット': 26, '価格': 27, '回復': 28, '背景': 29, '一蹴': 30, '批判': 31, '日本国': 32, '氏': 33, '紀': 34, '見城': 35, '囚': 36, '林': 37, '死刑': 38, '理由': 39, '発信': 40, '長男': 41, '作家': 42, '幻冬舎': 43, '式': 44, '賞': 45, '贈呈': 46, 'コラ': 47, 'デマ': 48, '傘': 49, '拡散': 50, '画像': 51, '否定': 52, '大統領': 53, '招待': 54, '握手': 55, 'IT': 56, '外資': 57, '情報': 58, '流出': 59, '規制': 60, 'ロボット': 61, '声': 62, '導入': 63, '現場': 64, '配膳': 65, 'ハムスター': 66, '動画': 67, '虐待': 68, 'もろ刃': 69, '剣': 70, '排除': 71, '為': 72, '米': 73, '華': 74, 'ソフト': 75, '判定': 76, '商談': 77, '良しあし': 78, '不毛': 79, '不謹慎': 80, '連鎖': 81, '%': 82, '24': 83, '出荷': 84, '最大': 85, '減': 86, 'チャン': 87, 'パオ': 88, '休止': 89, '活動': 90, 'SNS': 91, '人気': 92, '炎上': 93, '訓練': 94, '防災': 95, '安全ピン': 96, '撃退': 97, '痴漢': 98, '賛否': 

In [52]:
vec = dictionary.doc2bow(magachi[0])
print(vec)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


In [82]:
dense = list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0])

In [84]:
vec_all = []
for i in range(2000):
    vec = dictionary.doc2bow(magachi[i])
    vec_all.append(list(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0]))

In [79]:
dense_list = [list(matutils.corpus2dense(vec_all, num_terms=len(dictionary)).T[0])]

In [81]:
dense_list

[[1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [104]:
X = vec_all
Y = [0 if i<1000 else 1 for i in range(2000)]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

In [105]:
random_forest= ensemble.RandomForestClassifier(max_depth=2)
random_forest.fit(X_train, Y_train)
random_forest.score(X_test, Y_test)

/home/r-ei/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8325

In [101]:
len(Y)

2000